In [1]:
# from intcode import intcode, chain_intercodes, looping_intcodes

In [2]:
from collections import defaultdict

from typing import List


def intcode(program, inputs: List[int] = [], perpetual_input: bool = False,
            loop_mode: bool = False, print_outputs: bool = True, extend: int = 0,
            i: int = 0):
    assert len(program) > 0

    p = program.copy()
#     p.extend([0] * extend)

    relative_base = 0
    outputs = []

    while p[i] != 99:
        # Extract operation
        op = int(str(p[i])[-2:])
        # Extract modes after zero filling it & inverting
        modes = [int(char) for char in str(p[i])[:-2].zfill(3)][::-1]
#         print(f"db- i:{i} pi:{p[i]}")

        # Map variables/locations based on operation
        if op in [1, 2, 7, 8]:
            # Two parameters, 1 target
            variables = [
                i + 1 if modes[0] == 1 else p[i + 1] + relative_base if modes[0] == 2 else p[i + 1],
                i + 2 if modes[1] == 1 else p[i + 2] + relative_base if modes[1] == 2 else p[i + 2],
                i + 3 if modes[2] == 1 else p[i + 3] + relative_base if modes[2] == 2 else p[i + 3]
            ]
        elif op in [5, 6]:
            # Two parameters
            variables = [
                i + 1 if modes[0] == 1 else p[i + 1] + relative_base if modes[0] == 2 else p[i + 1],
                i + 2 if modes[1] == 1 else p[i + 2] + relative_base if modes[1] == 2 else p[i + 2]
            ]
        elif op in [3, 4, 9]:
            # One location?
            variables = [
                i + 1 if modes[0] == 1 else p[i + 1] + relative_base if modes[0] == 2 else p[i + 1]
            ]
#         elif op in [9]:
#             # One parameter
#             variables = [
#                 p[i + 1] if modes[0] == 1 else p[p[i + 1]] if modes[0] == 0 else p[
#                     p[i + 1] + relative_base],
#             ]

        # Perform operation
        if op == 1:  # Sum
            p[variables[2]] = p[variables[0]] + p[variables[1]]
            i += 4
        elif op == 2:  # Multiplication
            p[variables[2]] = p[variables[0]] * p[variables[1]]
            i += 4
        elif op == 3:  # Input
            p[variables[0]] = inputs[0] if perpetual_input and len(
                inputs) == 0 else inputs.pop(0)
            i += 2
        elif op == 4:  # Output
            outputs.append(p[variables[0]])
            if print_outputs:
                print(p[variables[0]])
            i += 2
            if loop_mode:
                return outputs, p.copy(), i
        elif op == 5:  # Jump-if-true
            i = p[variables[1]] if p[variables[0]] else i + 3
        elif op == 6:  # Jump-if-false
            i = i + 3 if p[variables[0]] else p[variables[1]]
        elif op == 7:  # Less-than
            p[variables[2]] = int(p[variables[0]] < p[variables[1]])
            i += 4
        elif op == 8:  # Equals
            p[variables[2]] = int(p[variables[0]] == p[variables[1]])
            i += 4
        elif op == 9:  # Relative base adjustment
            relative_base += p[variables[0]]
            i += 2
        else:  # ERROR!!
            print("Oops...")
            return None, None, -1

    if loop_mode:
        return None, None, -1
    return outputs, p, i


In [20]:
from typing import List


def intcode(program, inputs: List[int] = [], perpetual_input: bool = False,
            loop_mode: bool = False, print_outputs: bool = True,
            i: int = 0, relative_base=0):
    assert len(program) > 0

    p = program.copy()

    outputs = []

    while p[i] != 99:
        # Extract operation
        op = int(str(p[i])[-2:])
        # Extract modes after zero filling it & inverting
        modes = [int(char) for char in str(p[i])[:-2].zfill(3)][::-1]
#         print(f"db- i:{i} pi:{p[i]}")

        # Map variables/locations based on operation
        variables = [
            i + 1 if modes[0] == 1 else p[i + 1] + relative_base if modes[0] == 2 else p[i + 1],
            i + 2 if modes[1] == 1 else p[i + 2] + relative_base if modes[1] == 2 else p[i + 2],
            i + 3 if modes[2] == 1 else p[i + 3] + relative_base if modes[2] == 2 else p[i + 3]
        ]


        # Perform operation
        if op == 1:  # Sum
            p[variables[2]] = p[variables[0]] + p[variables[1]]
            i += 4
        elif op == 2:  # Multiplication
            p[variables[2]] = p[variables[0]] * p[variables[1]]
            i += 4
        elif op == 3:  # Input
            p[variables[0]] = inputs[0] if perpetual_input and len(
                inputs) == 0 else inputs.pop(0)
            i += 2
        elif op == 4:  # Output
            outputs.append(p[variables[0]])
            if print_outputs:
                print(p[variables[0]])
            i += 2
            if loop_mode:
                return outputs, p.copy(), i, relative_base
        elif op == 5:  # Jump-if-true
            i = p[variables[1]] if p[variables[0]] else i + 3
        elif op == 6:  # Jump-if-false
            i = i + 3 if p[variables[0]] else p[variables[1]]
        elif op == 7:  # Less-than
            p[variables[2]] = int(p[variables[0]] < p[variables[1]])
            i += 4
        elif op == 8:  # Equals
            p[variables[2]] = int(p[variables[0]] == p[variables[1]])
            i += 4
        elif op == 9:  # Relative base adjustment
            relative_base += p[variables[0]]
            i += 2
        else:  # ERROR!!
            print("Oops...")
            return None, None, -1, -1

    if loop_mode:
        return None, None, -1, -1
    return outputs, p, i, relative_base


In [21]:
with open('../inputs/11', 'r') as f:
    file_lines = f.readlines()
input_lines = [int(line.strip()) for line in file_lines[0].split(",")]

In [22]:
input_program = defaultdict(int, enumerate(input_lines))

In [9]:
panels = defaultdict(lambda: defaultdict(int))
orig_panels = panels.copy()
current_position = [0,0]

curr_i = 0
direction = 0
rbase = 0
program = input_program.copy()
# program.extend([0 for i in range(100000000)])
results = []
painted_panels = set()
while not curr_i == -1 and program is not None:
    x, y = current_position
    
    print (f"1-{x},{y} Instruction: {panels[x][y]}. i: {curr_i}")
    
    outputs, program, curr_i, rbase = intcode(program, [panels[x][y]], loop_mode=True, i=curr_i, relative_base=rbase, print_outputs=False)
    if outputs is not None:
        output = outputs[-1]
        results.append(output)

        if not panels[x][y] == output:
            painted_panels.add(tuple(current_position.copy()))
            panels[x][y] = output
    
    print (f"1-{x},{y} Result: {output}. i: {curr_i}")
    
    outputs, program, curr_i, rbase = intcode(program, [], loop_mode=True, i=curr_i, relative_base=rbase print_outputs=False)
    if outputs is not None:
        output = outputs[-1]
        results.append(output)

        direction = (direction + (1 if output else -1)) % 4

        if direction == 0:
            current_position[1] -= 1
        elif direction == 2:
            current_position[1] += 1
        elif direction == 1:
            current_position[0] += 1
        elif direction == 3:
            current_position[0] -= 1

    print (f"2-{x},{y} Result: {output}. Direction: {direction} New position: {current_position}. i: {curr_i}")
        
#     print(panels)

1-0,0 Instruction: 0. i: 0
1-0,0 Result: 1. i: 13
2-0,0 Result: 0. Direction: 3 New position: [-1, 0]. i: 15
1--1,0 Instruction: 0. i: 15
1--1,0 Result: 1. i: 27
2--1,0 Result: 1. Direction: 0 New position: [-1, -1]. i: 33
1--1,-1 Instruction: 0. i: 33
1--1,-1 Result: 1. i: 56
2--1,-1 Result: 0. Direction: 3 New position: [-2, -1]. i: 62
1--2,-1 Instruction: 0. i: 62
1--2,-1 Result: 1. i: 81
2--2,-1 Result: 1. Direction: 0 New position: [-2, -2]. i: 87
1--2,-2 Instruction: 0. i: 87
1--2,-2 Result: 1. i: 107
2--2,-2 Result: 1. Direction: 1 New position: [-1, -2]. i: 113
1--1,-2 Instruction: 0. i: 113
1--1,-2 Result: 1. i: 136
2--1,-2 Result: 0. Direction: 0 New position: [-1, -3]. i: 142
1--1,-3 Instruction: 0. i: 142
1--1,-3 Result: 1. i: 170
2--1,-3 Result: 1. Direction: 1 New position: [0, -3]. i: 176
1-0,-3 Instruction: 0. i: 176
1-0,-3 Result: 1. i: 199
2-0,-3 Result: 0. Direction: 0 New position: [0, -4]. i: 205
1-0,-4 Instruction: 0. i: 205
1-0,-4 Result: 1. i: 232
2-0,-4 Result:

2-0,11 Result: 1. Direction: 0 New position: [0, 10]. i: 87
1-0,10 Instruction: 1. i: 87
1-0,10 Result: 0. i: 107
2-0,10 Result: 0. Direction: 3 New position: [-1, 10]. i: 113
1--1,10 Instruction: 0. i: 113
1--1,10 Result: 1. i: 136
2--1,10 Result: 1. Direction: 0 New position: [-1, 9]. i: 142
1--1,9 Instruction: 1. i: 142
1--1,9 Result: 0. i: 170
2--1,9 Result: 1. Direction: 1 New position: [0, 9]. i: 176
1-0,9 Instruction: 1. i: 176
1-0,9 Result: 0. i: 199
2-0,9 Result: 1. Direction: 2 New position: [0, 10]. i: 205
1-0,10 Instruction: 0. i: 205
1-0,10 Result: 1. i: 232
2-0,10 Result: 0. Direction: 1 New position: [1, 10]. i: 238
1-1,10 Instruction: 1. i: 238
1-1,10 Result: 0. i: 254
2-1,10 Result: 1. Direction: 2 New position: [1, 11]. i: 260
1-1,11 Instruction: 1. i: 260
1-1,11 Result: 0. i: 276
2-1,11 Result: 0. Direction: 1 New position: [2, 11]. i: 282
1-2,11 Instruction: 0. i: 282
1-2,11 Result: 1. i: 27
2-2,11 Result: 1. Direction: 2 New position: [2, 12]. i: 33
1-2,12 Instruct

1-0,13 Result: 0. i: 81
2-0,13 Result: 1. Direction: 2 New position: [0, 14]. i: 87
1-0,14 Instruction: 0. i: 87
1-0,14 Result: 1. i: 107
2-0,14 Result: 1. Direction: 3 New position: [-1, 14]. i: 113
1--1,14 Instruction: 1. i: 113
1--1,14 Result: 0. i: 136
2--1,14 Result: 1. Direction: 0 New position: [-1, 13]. i: 142
1--1,13 Instruction: 1. i: 142
1--1,13 Result: 0. i: 170
2--1,13 Result: 1. Direction: 1 New position: [0, 13]. i: 176
1-0,13 Instruction: 0. i: 176
1-0,13 Result: 1. i: 199
2-0,13 Result: 1. Direction: 2 New position: [0, 14]. i: 205
1-0,14 Instruction: 1. i: 205
1-0,14 Result: 0. i: 232
2-0,14 Result: 1. Direction: 3 New position: [-1, 14]. i: 238
1--1,14 Instruction: 0. i: 238
1--1,14 Result: 1. i: 254
2--1,14 Result: 1. Direction: 0 New position: [-1, 13]. i: 260
1--1,13 Instruction: 0. i: 260
1--1,13 Result: 1. i: 276
2--1,13 Result: 0. Direction: 3 New position: [-2, 13]. i: 282
1--2,13 Instruction: 1. i: 282
1--2,13 Result: 0. i: 27
2--2,13 Result: 0. Direction: 2 

1--16,5 Result: 1. i: 81
2--16,5 Result: 1. Direction: 2 New position: [-16, 6]. i: 87
1--16,6 Instruction: 0. i: 87
1--16,6 Result: 1. i: 107
2--16,6 Result: 0. Direction: 1 New position: [-15, 6]. i: 113
1--15,6 Instruction: 0. i: 113
1--15,6 Result: 1. i: 136
2--15,6 Result: 1. Direction: 2 New position: [-15, 7]. i: 142
1--15,7 Instruction: 1. i: 142
1--15,7 Result: 0. i: 170
2--15,7 Result: 0. Direction: 1 New position: [-14, 7]. i: 176
1--14,7 Instruction: 0. i: 176
1--14,7 Result: 1. i: 199
2--14,7 Result: 1. Direction: 2 New position: [-14, 8]. i: 205
1--14,8 Instruction: 1. i: 205
1--14,8 Result: 0. i: 232
2--14,8 Result: 0. Direction: 1 New position: [-13, 8]. i: 238
1--13,8 Instruction: 1. i: 238
1--13,8 Result: 0. i: 254
2--13,8 Result: 0. Direction: 0 New position: [-13, 7]. i: 260
1--13,7 Instruction: 0. i: 260
1--13,7 Result: 1. i: 276
2--13,7 Result: 1. Direction: 1 New position: [-12, 7]. i: 282
1--12,7 Instruction: 0. i: 282
1--12,7 Result: 1. i: 27
2--12,7 Result: 1.

1-2,29 Instruction: 1. i: 62
1-2,29 Result: 0. i: 81
2-2,29 Result: 0. Direction: 0 New position: [2, 28]. i: 87
1-2,28 Instruction: 1. i: 87
1-2,28 Result: 0. i: 107
2-2,28 Result: 0. Direction: 3 New position: [1, 28]. i: 113
1-1,28 Instruction: 1. i: 113
1-1,28 Result: 0. i: 136
2-1,28 Result: 0. Direction: 2 New position: [1, 29]. i: 142
1-1,29 Instruction: 0. i: 142
1-1,29 Result: 1. i: 170
2-1,29 Result: 0. Direction: 1 New position: [2, 29]. i: 176
1-2,29 Instruction: 0. i: 176
1-2,29 Result: 1. i: 199
2-2,29 Result: 0. Direction: 0 New position: [2, 28]. i: 205
1-2,28 Instruction: 0. i: 205
1-2,28 Result: 1. i: 232
2-2,28 Result: 1. Direction: 1 New position: [3, 28]. i: 238
1-3,28 Instruction: 1. i: 238
1-3,28 Result: 0. i: 254
2-3,28 Result: 0. Direction: 0 New position: [3, 27]. i: 260
1-3,27 Instruction: 1. i: 260
1-3,27 Result: 0. i: 276
2-3,27 Result: 0. Direction: 3 New position: [2, 27]. i: 282
1-2,27 Instruction: 1. i: 282
1-2,27 Result: 0. i: 27
2-2,27 Result: 0. Dire

...

1--6,-12 Result: 0. i: 232
2--6,-12 Result: 0. Direction: 3 New position: [-7, -12]. i: 238
1--7,-12 Instruction: 0. i: 238
1--7,-12 Result: 1. i: 254
2--7,-12 Result: 0. Direction: 2 New position: [-7, -11]. i: 260
1--7,-11 Instruction: 1. i: 260
1--7,-11 Result: 0. i: 276
2--7,-11 Result: 1. Direction: 3 New position: [-8, -11]. i: 282
1--8,-11 Instruction: 1. i: 282
1--8,-11 Result: 0. i: 27
2--8,-11 Result: 0. Direction: 2 New position: [-8, -10]. i: 33
1--8,-10 Instruction: 1. i: 33
1--8,-10 Result: 0. i: 56
2--8,-10 Result: 1. Direction: 3 New position: [-9, -10]. i: 62
1--9,-10 Instruction: 1. i: 62
1--9,-10 Result: 0. i: 81
2--9,-10 Result: 0. Direction: 2 New position: [-9, -9]. i: 87
1--9,-9 Instruction: 1. i: 87
1--9,-9 Result: 0. i: 107
2--9,-9 Result: 0. Direction: 1 New position: [-8, -9]. i: 113
1--8,-9 Instruction: 1. i: 113
1--8,-9 Result: 0. i: 136
2--8,-9 Result: 1. Direction: 2 New position: [-8, -8]. i: 142
1--8,-8 Instruction: 1. i: 142
1--8,-8 Result: 0. i: 170
2

1--27,-20 Result: 0. i: 27
2--27,-20 Result: 0. Direction: 2 New position: [-27, -19]. i: 33
1--27,-19 Instruction: 0. i: 33
1--27,-19 Result: 1. i: 56
2--27,-19 Result: 0. Direction: 1 New position: [-26, -19]. i: 62
1--26,-19 Instruction: 0. i: 62
1--26,-19 Result: 1. i: 81
2--26,-19 Result: 0. Direction: 0 New position: [-26, -20]. i: 87
1--26,-20 Instruction: 0. i: 87
1--26,-20 Result: 1. i: 107
2--26,-20 Result: 1. Direction: 1 New position: [-25, -20]. i: 113
1--25,-20 Instruction: 1. i: 113
1--25,-20 Result: 0. i: 136
2--25,-20 Result: 0. Direction: 0 New position: [-25, -21]. i: 142
1--25,-21 Instruction: 0. i: 142
1--25,-21 Result: 1. i: 170
2--25,-21 Result: 1. Direction: 1 New position: [-24, -21]. i: 176
1--24,-21 Instruction: 1. i: 176
1--24,-21 Result: 0. i: 199
2--24,-21 Result: 0. Direction: 0 New position: [-24, -22]. i: 205
1--24,-22 Instruction: 0. i: 205
1--24,-22 Result: 1. i: 232
2--24,-22 Result: 0. Direction: 3 New position: [-25, -22]. i: 238
1--25,-22 Instruct

2--39,-7 Result: 1. Direction: 3 New position: [-40, -7]. i: 113
1--40,-7 Instruction: 0. i: 113
1--40,-7 Result: 1. i: 136
2--40,-7 Result: 0. Direction: 2 New position: [-40, -6]. i: 142
1--40,-6 Instruction: 0. i: 142
1--40,-6 Result: 1. i: 170
2--40,-6 Result: 1. Direction: 3 New position: [-41, -6]. i: 176
1--41,-6 Instruction: 0. i: 176
1--41,-6 Result: 1. i: 199
2--41,-6 Result: 0. Direction: 2 New position: [-41, -5]. i: 205
1--41,-5 Instruction: 0. i: 205
1--41,-5 Result: 1. i: 232
2--41,-5 Result: 1. Direction: 3 New position: [-42, -5]. i: 238
1--42,-5 Instruction: 0. i: 238
1--42,-5 Result: 1. i: 254
2--42,-5 Result: 1. Direction: 0 New position: [-42, -6]. i: 260
1--42,-6 Instruction: 0. i: 260
1--42,-6 Result: 1. i: 276
2--42,-6 Result: 1. Direction: 1 New position: [-41, -6]. i: 282
1--41,-6 Instruction: 1. i: 282
1--41,-6 Result: 0. i: 27
2--41,-6 Result: 0. Direction: 0 New position: [-41, -7]. i: 33
1--41,-7 Instruction: 0. i: 33
1--41,-7 Result: 1. i: 56
2--41,-7 Res

1--51,-8 Instruction: 0. i: 282
1--51,-8 Result: 1. i: 27
2--51,-8 Result: 0. Direction: 2 New position: [-51, -7]. i: 33
1--51,-7 Instruction: 1. i: 33
1--51,-7 Result: 0. i: 56
2--51,-7 Result: 1. Direction: 3 New position: [-52, -7]. i: 62
1--52,-7 Instruction: 0. i: 62
1--52,-7 Result: 1. i: 81
2--52,-7 Result: 0. Direction: 2 New position: [-52, -6]. i: 87
1--52,-6 Instruction: 0. i: 87
1--52,-6 Result: 1. i: 107
2--52,-6 Result: 0. Direction: 1 New position: [-51, -6]. i: 113
1--51,-6 Instruction: 0. i: 113
1--51,-6 Result: 1. i: 136
2--51,-6 Result: 0. Direction: 0 New position: [-51, -7]. i: 142
1--51,-7 Instruction: 0. i: 142
1--51,-7 Result: 1. i: 170
2--51,-7 Result: 0. Direction: 3 New position: [-52, -7]. i: 176
1--52,-7 Instruction: 1. i: 176
1--52,-7 Result: 0. i: 199
2--52,-7 Result: 1. Direction: 0 New position: [-52, -8]. i: 205
1--52,-8 Instruction: 0. i: 205
1--52,-8 Result: 1. i: 232
2--52,-8 Result: 1. Direction: 1 New position: [-51, -8]. i: 238
1--51,-8 Instruct

TypeError: object of type 'NoneType' has no len()

In [10]:
len(painted_panels)

1964

In [11]:
## part 2

In [5]:
panels = defaultdict(lambda: defaultdict(int))
orig_panels = panels.copy()
current_position = [0,0]

panels[0][0] = 1
panels

defaultdict(<function __main__.<lambda>()>, {0: defaultdict(int, {0: 1})})

In [9]:
panels[1]

defaultdict(int, {0: 0})

In [25]:
panels = defaultdict(lambda: defaultdict(int))
panels[0][0] = 1
orig_panels = panels.copy()
current_position = [0,0]

curr_i = 0
direction = 0
rbase = 0
program = input_program.copy()
results = []
painted_panels = set()
while not curr_i == -1 and program is not None:
    x, y = current_position
    
    print (f"1-{x},{y} Instruction: {panels[x][y]}. i: {curr_i}")
    
    outputs, program, curr_i, rbase = intcode(program, [panels[x][y]], loop_mode=True, i=curr_i, relative_base=rbase,print_outputs=False)
    if outputs is not None:
        output = outputs[-1]
        results.append(output)

        if not panels[x][y] == output:
            painted_panels.add(tuple(current_position.copy()))
            panels[x][y] = output
    
    print (f"1-{x},{y} Result: {output}. i: {curr_i}")
    
    outputs, program, curr_i, rbase = intcode(program, [], loop_mode=True, i=curr_i, relative_base=rbase,print_outputs=False)
    if outputs is not None:
        output = outputs[-1]
        results.append(output)

        direction = (direction + (1 if output else -1)) % 4

        if direction == 0:
            current_position[1] -= 1
        elif direction == 2:
            current_position[1] += 1
        elif direction == 1:
            current_position[0] += 1
        elif direction == 3:
            current_position[0] -= 1

    print (f"2-{x},{y} Result: {output}. Direction: {direction} New position: {current_position}. i: {curr_i}")
        
#     print(panels)

1-0,0 Instruction: 1. i: 0
1-0,0 Result: 0. i: 306
2-0,0 Result: 1. Direction: 1 New position: [1, 0]. i: 308
1-1,0 Instruction: 0. i: 308
1-1,0 Result: 1. i: 460
2-1,0 Result: 1. Direction: 2 New position: [1, 1]. i: 466
1-1,1 Instruction: 0. i: 466
1-1,1 Result: 1. i: 460
2-1,1 Result: 0. Direction: 1 New position: [2, 1]. i: 466
1-2,1 Instruction: 0. i: 466
1-2,1 Result: 0. i: 460
2-2,1 Result: 0. Direction: 0 New position: [2, 0]. i: 466
1-2,0 Instruction: 0. i: 466
1-2,0 Result: 1. i: 460
2-2,0 Result: 1. Direction: 1 New position: [3, 0]. i: 466
1-3,0 Instruction: 0. i: 466
1-3,0 Result: 1. i: 460
2-3,0 Result: 1. Direction: 2 New position: [3, 1]. i: 466
1-3,1 Instruction: 0. i: 466
1-3,1 Result: 0. i: 460
2-3,1 Result: 0. Direction: 1 New position: [4, 1]. i: 466
1-4,1 Instruction: 0. i: 466
1-4,1 Result: 0. i: 460
2-4,1 Result: 0. Direction: 0 New position: [4, 0]. i: 466
1-4,0 Instruction: 0. i: 466
1-4,0 Result: 1. i: 460
2-4,0 Result: 1. Direction: 1 New position: [5, 0]. i

TypeError: object of type 'NoneType' has no len()

In [80]:
res = [[i for i,item in row.items()] for j,row in panels.items()]

In [26]:
min_j = min(panels.keys())
min_j
max_j = max(panels.keys())

In [27]:
min_i = min([min(items.keys()) for items in panels.values()])
min_i
max_i = max([max(items.keys()) for items in panels.values()])

In [28]:
res = {j+min_j: {i+min_i: item for i,item in row.items()} for j,row in panels.items()}

In [29]:
array_result = ""
ascii_map = ["█", "░"]

for i in range(min_i, max_i + 1):
    for j in range(min_j, max_j + 1):
        array_result += f"{ascii_map[panels[j][i]]}"
    array_result += "\n"

In [30]:
print(array_result)

█░░░░█░██░█░░░░█░██░██░░██░░░░█░░░██░██░███
█░████░█░██░████░█░██░██░█░████░██░█░█░████
█░░░██░░███░░░██░░███░████░░░██░██░█░░█████
█░████░█░██░████░█░██░████░████░░░██░█░████
█░████░█░██░████░█░██░██░█░████░█░██░█░████
█░████░██░█░░░░█░██░██░░██░████░██░█░██░███



In [78]:
res

{67: {28: 0,
  25: 1,
  24: 1,
  23: 1,
  22: 1,
  26: 0,
  27: 0,
  37: 0,
  38: 0,
  39: 1,
  40: 1,
  41: 0,
  42: 0,
  43: 0,
  49: 0,
  48: 0,
  47: 0,
  46: 0,
  55: 0,
  54: 1,
  56: 0,
  57: 0,
  58: 1,
  61: 0,
  62: 1,
  60: 1,
  59: 1,
  53: 1,
  51: 0,
  50: 1,
  52: 1,
  36: 1,
  35: 1,
  34: 1,
  31: 0,
  30: 0,
  33: 1,
  44: 0,
  45: 1,
  32: 0,
  29: 0,
  19: 1,
  18: 1},
 66: {28: 0,
  27: 1,
  26: 0,
  25: 0,
  29: 0,
  32: 1,
  33: 0,
  34: 0,
  36: 1,
  37: 1,
  38: 1,
  39: 1,
  42: 0,
  43: 1,
  41: 0,
  40: 0,
  48: 0,
  47: 1,
  49: 0,
  46: 1,
  44: 0,
  45: 1,
  54: 1,
  55: 0,
  56: 0,
  57: 0,
  58: 1,
  59: 0,
  60: 1,
  62: 1,
  63: 1,
  61: 0,
  53: 1,
  50: 1,
  52: 1,
  51: 1,
  35: 1,
  31: 0,
  30: 0,
  22: 1,
  21: 1,
  20: 0,
  19: 0,
  18: 1,
  17: 1},
 65: {27: 1,
  26: 0,
  29: 1,
  30: 1,
  33: 1,
  32: 0,
  34: 1,
  31: 1,
  39: 0,
  40: 1,
  37: 1,
  36: 1,
  43: 0,
  44: 1,
  42: 0,
  41: 1,
  38: 1,
  45: 0,
  46: 1,
  49: 0,
  48: 0,
  47:

In [73]:
res_2 = [[item for item in row.values()] for row in res.values()]

In [67]:
res_2

[[0,
  -3,
  -4,
  -5,
  -6,
  -2,
  -1,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  21,
  20,
  19,
  18,
  27,
  26,
  28,
  29,
  30,
  33,
  34,
  32,
  31,
  25,
  23,
  22,
  24,
  8,
  7,
  6,
  3,
  2,
  5,
  16,
  17,
  4,
  1,
  -9,
  -10],
 [0,
  -1,
  -2,
  -3,
  1,
  4,
  5,
  6,
  8,
  9,
  10,
  11,
  14,
  15,
  13,
  12,
  20,
  19,
  21,
  18,
  16,
  17,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  34,
  35,
  33,
  25,
  22,
  24,
  23,
  7,
  3,
  2,
  -6,
  -7,
  -8,
  -9,
  -10,
  -11],
 [-1,
  -2,
  1,
  2,
  5,
  4,
  6,
  3,
  11,
  12,
  9,
  8,
  15,
  16,
  14,
  13,
  10,
  17,
  18,
  21,
  20,
  19,
  25,
  26,
  27,
  29,
  28,
  30,
  31,
  32,
  33,
  34,
  35,
  24,
  7,
  -3,
  -4,
  0,
  -7,
  -6,
  -5,
  -8,
  -9,
  -10,
  -11],
 [-4,
  -5,
  -6,
  -3,
  10,
  11,
  12,
  13,
  14,
  15,
  26,
  27,
  28,
  29,
  30,
  32,
  31,
  22,
  21,
  20,
  19,
  23,
  24,
  25,
  18,
  17,
  9,
  6,
  5,
  4,
  3,
  2,
  1,
  7,
  8,
  16,
  0,
  -1,
  -2],

In [77]:
res

{67: {28: 0,
  25: 1,
  24: 1,
  23: 1,
  22: 1,
  26: 0,
  27: 0,
  37: 0,
  38: 0,
  39: 1,
  40: 1,
  41: 0,
  42: 0,
  43: 0,
  49: 0,
  48: 0,
  47: 0,
  46: 0,
  55: 0,
  54: 1,
  56: 0,
  57: 0,
  58: 1,
  61: 0,
  62: 1,
  60: 1,
  59: 1,
  53: 1,
  51: 0,
  50: 1,
  52: 1,
  36: 1,
  35: 1,
  34: 1,
  31: 0,
  30: 0,
  33: 1,
  44: 0,
  45: 1,
  32: 0,
  29: 0,
  19: 1,
  18: 1},
 66: {28: 0,
  27: 1,
  26: 0,
  25: 0,
  29: 0,
  32: 1,
  33: 0,
  34: 0,
  36: 1,
  37: 1,
  38: 1,
  39: 1,
  42: 0,
  43: 1,
  41: 0,
  40: 0,
  48: 0,
  47: 1,
  49: 0,
  46: 1,
  44: 0,
  45: 1,
  54: 1,
  55: 0,
  56: 0,
  57: 0,
  58: 1,
  59: 0,
  60: 1,
  62: 1,
  63: 1,
  61: 0,
  53: 1,
  50: 1,
  52: 1,
  51: 1,
  35: 1,
  31: 0,
  30: 0,
  22: 1,
  21: 1,
  20: 0,
  19: 0,
  18: 1,
  17: 1},
 65: {27: 1,
  26: 0,
  29: 1,
  30: 1,
  33: 1,
  32: 0,
  34: 1,
  31: 1,
  39: 0,
  40: 1,
  37: 1,
  36: 1,
  43: 0,
  44: 1,
  42: 0,
  41: 1,
  38: 1,
  45: 0,
  46: 1,
  49: 0,
  48: 0,
  47:

In [39]:
min(panels.keys())

-67

In [74]:
def print_image(image_arrays, ascii=False):
    ascii_map = ["█", "░"]
    output = ""
    for array in image_arrays:
        for item in array:
            output+= f"{str(item) if not ascii else ascii_map[item]}"
        output += "\n"
    return output

In [76]:
print(print_image(res_2, ascii=True))

█░░░░████░░████████░██░█░░░░█░░░░░██░█░██░░
█░███░██░░░░█░███░█░█░░███░█░░░█░░░░░██░░██░░
░█░░░█░░█░░░█░█░░█░██░░░░█░░░░░░█░░░░░░█░█░█░
██░████░██░████░█░░░░█░░░██░██░░░░███░░
░░██░░░███░░█░░█░███░░█░███░░██░██
██░░██░███░░█░░█░█░░░█░██░░░░██░░░░░░░███░░░░██░█
░░██░██░██████░░░░███░░░░░█████░██████░██░█░░░█░
███░███░░░░░█░░░█░░██░█████████░██░█░░██░░░░█░░
░░░░█░████░░███░░█░█░░██░░██░█░░░█░░░░░█░█░░░░██
██░██░░░███░░█░░░█░██░█░░░░░░░░░░░██░░█░░█████░░░░
██░░█░███░░░█░░░███░░░█░░███░░██░░██░░░░█░░██░██░
░░██░░░░░█░█░░░███░░░██░██░░█░█░░░░░████░░░███░░
█░█████████████░░███░█░█░█░████░░░░██░█░█░░░░░
██░█░██░██░░░░░██░█░░█░░░░░██░██░░░██░░█░░
░███░░░██░█░░░░█░░░█░░░░█░░█░█░█░░█░░░░
████░██░██░████░██░░███░█░░███░░░█
░░░░░█░█░░██░█░░██░░██░░█░░░░█░░█░█
█░████░█░░█░███░█░█████░░█░░░░░░███░
░████░█████░░░░░░░███░██░█░█░░█░░░░█
█░█░█░███░████░░░░░░█░░█░░█░░█░░
░░░░░░░░░░░█░██░██░░█░██░░░░
███░█░█░░████░████░██░░█░█░
█░░░░██░███░░█░██░░█░█░█░░██░░░░░░
░███░█░░░██░░░██░█░█░█░█░░░░█░██░
██░░█░███